In [20]:
import numpy as np
import torch, queue
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.utils.data import Dataset
from src.transforms import *
from torch_geometric.data import Data, HeteroData

In [43]:
class TerrainHeteroData(HeteroData):
    def __inc__(self, key, value, *args, **kwargs):
        if key == 'src':
            return self['real'].x.size(0)
        if key == 'tar':
            return self['real'].x.size(0)
        return super().__inc__(key, value, *args, **kwargs)

class TerrainPatchesData(Data):
    def __inc__(self, key, value, *args, **kwargs):
        if key == 'src':
            return self.x.size(0)
        if key == 'tar':
            return self.x.size(0)
        return super().__inc__(key, value, *args, **kwargs)
    

In [44]:
def add_virtual_node(data):
    hetero_data = TerrainHeteroData()
    sz_features = data.x.size()[1]
    hetero_data.src = data.src
    hetero_data.tar = data.tar
    hetero_data['real'].x = data.x.double()
    hetero_data['real', 'e1', 'real'].edge_index = data.edge_index

    vn = torch.zeros(size = (1, sz_features), dtype=torch.double )
    hetero_data['vn'].x = vn
    vn_edge_index = [[], []]
    for i in range(data.x.size()[0]):
        vn_edge_index[0].append(0)
        vn_edge_index[1].append(i)
    hetero_data['vn', 'e2', 'real'].edge_index = torch.tensor(vn_edge_index, dtype=torch.long)

    return hetero_data

In [45]:
x1 = torch.randn(3, 16)  
ei_1 = torch.tensor([
    [0, 1, 0, 2],
    [1, 0, 2, 0],
])

x2 = torch.randn(3, 16)  
ei_2 = torch.tensor([
    [0, 1, 0, 2],
    [1, 0, 2, 0],
])

In [46]:
data0 = TerrainPatchesData(x=x1, edge_index = ei_1, src =1, tar=2)
data1 = TerrainPatchesData(x=x2, edge_index = ei_2, src = 0, tar=1)
vn1 = add_virtual_node(data0)
vn2 = add_virtual_node(data1)
lst = [data0, data1]
lst_vn = [vn1, vn2]
print(vn1)

TerrainHeteroData(
  src=1,
  real={ x=[3, 16] },
  vn={ x=[1, 16] },
  (real, e1, real)={ edge_index=[2, 4] },
  (vn, e2, real)={ edge_index=[2, 3] }
)


In [47]:
loader = DataLoader(lst_vn, batch_size=2, follow_batch = ['src'])
batch = next(iter(loader))
print(batch['vn', 'e2', 'real'])
print(batch['src'])

{'edge_index': tensor([[0, 0, 0, 1, 1, 1],
        [0, 1, 2, 3, 4, 5]])}
tensor([1, 3])


In [29]:
data0 = TerrainPatchesData(x = data[0].x, edge_index = data[0].edge_index, src=np.int64(1), tar=np.int64(2))
data1 = TerrainPatchesData(x = data[1].x, edge_index = data[1].edge_index, src=np.int64(0), tar=np.int64(0))
lst = [data0, data1]

In [30]:
loader = DataLoader(lst, batch_size=2, follow_batch=[ 'src', 'tar'])
batch = next(iter(loader))
print(batch.src)

[1, 0]
